# Streaming
This notebook covers functionality related to streaming.

For more information, see:

- [Streaming with LCEL](https://js.langchain.com/docs/expression_language/interface#stream)
- [Streaming for Agents](https://js.langchain.com/docs/modules/agents/how_to/streaming)

## Basic Streaming

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { StringOutputParser } from "@langchain/core/output_parsers";

In [2]:
const prompt = ChatPromptTemplate.fromTemplate("Tell me a joke about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const parser = new StringOutputParser();
const chain = prompt.pipe(model).pipe(parser);

In [3]:
for await (const step of await chain.stream({
  topic: "bears",
})) {
  console.log(step);
}


Why
 don
't
 bears
 wear
 shoes
?


Because
 they
 have
 bear
 feet
!



## Streaming with RunnableParallel

In [5]:
import { RunnableParallel } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromTemplate("Tell me a joke about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const parser = new StringOutputParser();
const chain1 = prompt.pipe(model).pipe(parser);

In [6]:
const prompt = ChatPromptTemplate.fromTemplate("Write me a poem about {topic}");
const model = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});
const parser = new StringOutputParser();
const chain2 = prompt.pipe(model).pipe(parser);

In [7]:
const parallelChain = new RunnableParallel({
  steps: {
    joke: chain1,
    poem: chain2
  }
});

for await (const step of await parallelChain.stream({
  topic: "bears",
})) {
  console.log(step);
}

{ joke: "" }
{ joke: "Why" }
{ joke: " don" }
{ joke: "'t" }
{ joke: " bears" }
{ joke: " wear" }
{ joke: " shoes" }
{ poem: "" }
{ poem: "In" }
{ poem: " the" }
{ poem: " heart" }
{ joke: "?\n\n" }
{ joke: "Because" }
{ joke: " they" }
{ joke: " have" }
{ joke: " bear" }
{ joke: " feet" }
{ joke: "!" }
{ joke: "" }
{ poem: " of" }
{ poem: " the" }
{ poem: " wild" }
{ poem: "," }
{ poem: " where" }
{ poem: " nature" }
{ poem: " thr" }
{ poem: "ives" }
{ poem: ",\n" }
{ poem: "Ro" }
{ poem: "ams" }
{ poem: " a" }
{ poem: " majestic" }
{ poem: " creature" }
{ poem: "," }
{ poem: " with" }
{ poem: " strength" }
{ poem: " and" }
{ poem: " drive" }
{ poem: ".\n" }
{ poem: "With" }
{ poem: " fur" }
{ poem: " of" }
{ poem: " gold" }
{ poem: "," }
{ poem: " or" }
{ poem: " black" }
{ poem: " as" }
{ poem: " night" }
{ poem: ",\n" }
{ poem: "B" }
{ poem: "ears" }
{ poem: " command" }
{ poem: " the" }
{ poem: " forest" }
{ poem: "," }
{ poem: " their" }
{ poem: " realm" }
{ poem: " of" }
{ poem:

In [9]:
let result = {};
for await (const step of await parallelChain.stream({
  topic: "bears",
})) {
  for (const [k, v] of Object.entries(step)) {
    result[k] = result[k] || "";
    result[k] += v;
  }
  console.log(result);
}

{ poem: "" }
{ poem: "In" }
{ poem: "In the" }
{ poem: "In the heart" }
{ poem: "In the heart of" }
{ poem: "In the heart of the" }
{ poem: "In the heart of the wild" }
{ poem: "In the heart of the wild," }
{ poem: "In the heart of the wild, where" }
{ poem: "In the heart of the wild, where nature" }
{ poem: "In the heart of the wild, where nature thr" }
{ poem: "In the heart of the wild, where nature thrives" }
{ poem: "In the heart of the wild, where nature thrives,\n" }
{ poem: "In the heart of the wild, where nature thrives,\nRo" }
{ poem: "In the heart of the wild, where nature thrives,\nRoams" }
{ poem: "In the heart of the wild, where nature thrives,\nRoams a" }
{
  poem: "In the heart of the wild, where nature thrives,\nRoams a majestic"
}
{
  poem: "In the heart of the wild, where nature thrives,\nRoams a majestic creature"
}
{
  poem: "In the heart of the wild, where nature thrives,\nRoams a majestic creature",
  joke: ""
}
{
  poem: "In the heart of the wild, where nature th

## Stream Log

In [14]:
import { TavilySearchAPIRetriever } from "@langchain/community/retrievers/tavily_search_api";
import { formatDocumentsAsString } from "langchain/util/document";
import { RunnablePassthrough } from "@langchain/core/runnables";

const retriever = new TavilySearchAPIRetriever();

const prompt = ChatPromptTemplate.fromTemplate(`Answer the question based only on the context provided:

Context: {context}

Question: {question}`);

const question = "what is langsmith"

const chain = prompt.pipe(model).pipe(parser);

const retrievalChain = RunnablePassthrough.assign({
  context: async (i: { question: string }) =>
  formatDocumentsAsString(await retriever.withConfig({ runName: "Docs" }).invoke(question))
}).withConfig({ runName: "passthrough" }).pipe(chain);

for await (const step of await retrievalChain.streamLog({
  question,
})) {
  console.log(step);
}

RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "d8b46c3f-e433-4ce8-a614-cee973b2cd66",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}


RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/passthrough",
      value: {
        id: "391acce4-0380-486b-8640-e660ea9868eb",
        name: "passthrough",
        type: "chain",
        tags: [ "seq:step:1" ],
        metadata: {},
        start_time: "2024-01-08T20:40:38.347Z",
        streamed_output_str: [],
        final_output: undefined,
        end_time: undefined
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/RunnableMap",
      value: {
        id: "98715365-bcf8-4342-9b3d-9f52a741e63f",
        name: "RunnableMap",
        type: "chain",
        tags: [],
        metadata: {},
        start_time: "2024-01-08T20:40:38.466Z",
        streamed_output_str: [],
        final_output: undefined,
        end_time: undefined
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/RunnableLambda",
      value: {
        id: "a1bb7fb4-ec7b-48e8-87ab-5f1677ca6d60",
        name: "RunnableLambda",
     

In [15]:
for await (const step of await retrievalChain.streamLog(
  {
    question,
  },
  {},
  {
    includeNames: ["Docs"]
  }
)) {
  console.log(step);
}

RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "a0dbe6b3-f2e8-4428-864a-2fdbf27c4aa9",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: "" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: "Lang" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: "Smith" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " is" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " a" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " platform" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " that" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " offers" } ]
}
RunLogPatch {
  ops: [ { op: "add", path: "/streamed_output/-", value: " features" } ]
}
RunL

## Agents

### Stream Actions

In [16]:
import { pull } from "langchain/hub";
import { createOpenAIFunctionsAgent, AgentExecutor } from "langchain/agents";
import { ChatOpenAI } from "@langchain/openai";
import { TavilySearchResults } from "@langchain/community/tools/tavily_search";

// Get the prompt to use - you can modify this!
const prompt = await pull<ChatPromptTemplate>("hwchase17/openai-functions-agent");

const search = new TavilySearchResults();
const tools = [search];

const llm = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
});

const agent = await createOpenAIFunctionsAgent({
  llm,
  tools,
  prompt,
});
const agentExecutor = new AgentExecutor({ agent, tools });

In [18]:
for await (const chunk of await agentExecutor.stream({
  input: "what is the weather in SF and then LA"
})) {
  console.log(chunk);
  console.log("---------")
}

{
  intermediateSteps: [
    {
      action: {
        tool: "tavily_search_results_json",
        toolInput: { input: "weather in San Francisco" },
        log: 'Invoking "tavily_search_results_json" with {\n' +
          '  "input": "weather in San Francisco"\n' +
          "}\n",
        messageLog: [ [AIMessage] ]
      },
      observation: '[{"title":"San Francisco weather in August 2024 | Weather25.com","url":"https://www.weather25.com/no'... 2308 more characters
    }
  ]
}
---------
{
  intermediateSteps: [
    {
      action: {
        tool: "tavily_search_results_json",
        toolInput: { input: "weather in Los Angeles" },
        log: 'Invoking "tavily_search_results_json" with {\n' +
          '  "input": "weather in Los Angeles"\n' +
          "}\n",
        messageLog: [ [AIMessage] ]
      },
      observation: '[{"title":"Cold Weather Expected in Los Angeles Today - 8 January 2024 - inkl","url":"https://www.in'... 2109 more characters
    }
  ]
}
---------
{
  output

### Stream Tokens

In [19]:
const llm = new ChatOpenAI({
  modelName: "gpt-3.5-turbo",
  temperature: 0,
  streaming: true,
});
const agent = await createOpenAIFunctionsAgent({
  llm,
  tools,
  prompt,
});
const agentExecutor = new AgentExecutor({ agent, tools });

In [20]:
for await (const step of await agentExecutor.streamLog(
  {
    input: "what is the weather in SF"
  },
  {},
  {
    includeNames: ["ChatOpenAI"]
  }
)) {
  console.log(step);
}

RunLogPatch {
  ops: [
    {
      op: "replace",
      path: "",
      value: {
        id: "42da378f-98f9-4bab-8ae3-722a3a77e42c",
        streamed_output: [],
        final_output: undefined,
        logs: {}
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI",
      value: {
        id: "37237010-0b65-4404-9e65-1ea23ea3054d",
        name: "ChatOpenAI",
        type: "llm",
        tags: [ "seq:step:3" ],
        metadata: {},
        start_time: "2024-01-08T20:45:21.247Z",
        streamed_output_str: [],
        final_output: undefined,
        end_time: undefined
      }
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI/streamed_output_str/-",
      value: ""
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI/streamed_output_str/-",
      value: ""
    }
  ]
}
RunLogPatch {
  ops: [
    {
      op: "add",
      path: "/logs/ChatOpenAI/streamed_output_str/-",
  